In [1]:
import tensorflow as tf

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [6]:
import pandas  as pd 
import numpy as np
import datatable as dt

In [10]:
from xgboost import XGBClassifier
from sklearn.model_selection import  train_test_split
from sklearn.model_selection  import StratifiedKFold
from sklearn.metrics import roc_curve, auc
import optuna


In [11]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [12]:
train = dt.fread('train.csv').to_pandas().drop('id', axis=1)
train  = reduce_memory_usage(train)

Mem. usage decreased to 501.63 Mb (73.2% reduction)


In [14]:
test = dt.fread('test.csv').to_pandas().drop('id', axis=1)
test = reduce_memory_usage(test)


Mem. usage decreased to 250.34 Mb (73.3% reduction)


In [15]:
sub = dt.fread('sample_submission.csv').to_pandas()
sub = reduce_memory_usage(sub)

Mem. usage decreased to 2.86 Mb (50.0% reduction)


In [17]:
bool_cols_train = []
for i , col in enumerate(train.columns):
    if train[col].dtype == bool:
        bool_cols_train.append(i)

In [18]:
bool_cols_test = []
for i , col in enumerate(test.columns):
    if train[col].dtype == bool:
        bool_cols_test.append(i)

In [19]:
train.iloc[:, bool_cols_train] = train.iloc[:, bool_cols_train].astype(int)
test.iloc[:, bool_cols_test] = test.iloc[:, bool_cols_test].astype(int)

In [20]:
print("Train set shape", train.shape, "\n", "Test set shape", test.shape)

Train set shape (1000000, 286) 
 Test set shape (500000, 285)


In [22]:
test.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284
0,0.178223,0.435547,0.010231,0.202026,0.390137,0.324219,0.221680,0.738770,0.582520,0.343750,...,1,0,0,0,0,0,1,1,1,0
1,0.181274,0.476562,0.022415,0.283203,0.598145,0.349609,0.283447,0.721680,0.269043,0.208374,...,0,0,0,0,0,0,0,0,0,0
2,0.159668,0.451172,0.259766,0.365234,0.594727,0.413574,0.249268,0.642578,0.411133,0.246948,...,0,0,0,0,0,0,1,0,0,0
3,0.182373,0.520996,0.095337,0.327637,0.741699,0.358643,0.270020,0.601562,0.297852,0.252930,...,0,0,0,0,0,1,1,0,0,0
4,0.229370,0.336426,0.023514,0.301025,0.668945,0.481689,0.545898,0.667969,0.545898,0.202759,...,0,0,0,0,1,0,0,1,0,0


In [23]:
train.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f276,f277,f278,f279,f280,f281,f282,f283,f284,target
0,0.205933,0.410889,0.176758,0.223633,0.423584,0.476074,0.413574,0.611816,0.534668,0.147339,...,0,1,0,0,0,0,0,0,0,1
1,0.181030,0.473145,0.011734,0.213623,0.619629,0.441650,0.230347,0.686035,0.281982,0.238525,...,0,1,0,0,0,0,0,0,0,1
2,0.182617,0.307373,0.325928,0.207153,0.605469,0.309814,0.493408,0.750977,0.536133,0.286865,...,0,0,0,1,1,0,0,0,0,1
3,0.180298,0.494629,0.008369,0.223633,0.760742,0.439209,0.432129,0.776367,0.483887,0.260986,...,0,0,0,0,1,0,0,0,0,1
4,0.177124,0.495605,0.014259,0.548828,0.625488,0.562500,0.117188,0.561035,0.077087,0.158325,...,0,1,1,0,1,0,0,1,0,1


In [26]:
X = train.drop('target', axis=1).copy()
y = train['target'].copy()
X_test = test.copy()

del train
del test

In [27]:
X['std'] = X.std(axis=1)
X['min'] = X.min(axis=1)
X['max'] = X.max(axis=1)

X_test['std'] = X_test.std(axis=1)
X_test['min'] = X_test.min(axis=1)
X_test['max'] = X_test.max(axis=1)

In [28]:
params = {
    'max_depth': 6,
    'n_estimators': 9550,
    'learning_rate': 0.007379718158350149,
    'subsample': 0.7,
    'colsample_bytree': 0.2,
    'colsample_bylevel': 0.6000000000000001,
    'min_child_weight': 56.41980735551558,
    'reg_lambda': 75.56651890088857,
    'reg_alpha': 0.11766857055687065,
    'gamma': 0.6407923221122686
    }

In [29]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

preds = []
scores = []

for fold, (idx_train, idx_valid) in enumerate(kf.split(X, y)):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
    model = XGBClassifier(**params,
                            booster= 'gbtree',
                            eval_metric = 'auc',
                            tree_method= 'gpu_hist',
                            predictor="gpu_predictor",
                            use_label_encoder=False)
    
    model.fit(X_train,y_train,
              eval_set=[(X_valid,y_valid)],
              early_stopping_rounds=100,
              verbose=False)
    
    pred_valid = model.predict_proba(X_valid)[:,1]
    fpr, tpr, _ = roc_curve(y_valid, pred_valid)
    score = auc(fpr, tpr)
    scores.append(score)
    
    print(f"Fold: {fold + 1} Score: {score}")
    print('||'*40)
    
    test_preds = model.predict_proba(X_test)[:,1]
    preds.append(test_preds)  
        
print(f"Overall Validation Score: {np.mean(scores)}")

Fold: 1 Score: 0.8595136137891262
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Fold: 2 Score: 0.8558509507152235
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Fold: 3 Score: 0.8564435595857822
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Fold: 4 Score: 0.8576236651265778
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Fold: 5 Score: 0.8567778795833604
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Fold: 6 Score: 0.8570875093529269
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Overall Validation Score: 0.8572161963588328


In [30]:
predictions = np.mean(np.column_stack(preds),axis=1)

sub['target'] = predictions
sub.to_csv('submission.csv', index=False)
sub.head()

,id,target
0,1000000,0.732030
1,1000001,0.232964
2,1000002,0.905649
3,1000003,0.836125
4,1000004,0.256379
